In [ ]:
from pyproj import Transformer
from colorcet import glasbey
import cuxfilter
from cuxfilter import charts

dbscan = cuml.DBSCAN(eps = 8150, min_samples = 13)

infected_df = gdf[gdf['infected'] == 1].reset_index(drop=True)
infected_df['cluster'] = dbscan.fit_predict(
    infected_df[['northing', 'easting']]
)

print(infected_df['cluster'].value_counts().sort_index())

transformer = Transformer.from_crs("epsg:27700", "epsg:3857", always_xy=True)

pdf = infected_df.to_pandas()
pdf["wm_easting"], pdf["wm_northing"] = transformer.transform(
    pdf["easting"].values,
    pdf["northing"].values,
)

pdf["weight"] = 1

infected_df = cudf.from_pandas(pdf)

cux_df = cuxfilter.DataFrame.from_dataframe(infected_df)

cluster_vals = (
    infected_df["cluster"]
    .to_pandas()
    .sort_values()
    .unique()
)
n_clusters = len(cluster_vals)
palette_clusters = list(glasbey[:n_clusters])

clusters_chart = charts.scatter(
    x="wm_easting",
    y="wm_northing",
    aggregate_col="cluster",
    aggregate_fn="mean",
    tile_provider="CartoLight",
    pixel_shade_type="linear",
    color_palette=palette_clusters,
    point_size=4,
    point_shape="circle",
    title="Infected people clusters",
)

density_chart = charts.heatmap(
    x="wm_easting",
    y="wm_northing",
    aggregate_col="weight",
    aggregate_fn="sum",
    title="Infected People Density",
)

cluster_multi_select = charts.multi_select("cluster")

dashboard = cux_df.dashboard(
    charts=[clusters_chart, density_chart],
    sidebar=[cluster_multi_select],
    layout=cuxfilter.layouts.double_feature,
    theme=cuxfilter.themes.dark,
    title="Dashboard",
    data_size_widget=True,
)

dashboard.app()